In [1]:
import pandas as pd
import seaborn as sns

%load_ext google.cloud.bigquery

# Channel KPI Version

In [4]:
params = {
    'report_start_date': '2022-10-01',
    'report_end_date': '2022-10-31'
}

In [5]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Unsubs_v4_Oct22_test` AS (
    SELECT DISTINCT adobe_tracking_id
    FROM  `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Unsubs`
    WHERE first_unsub_date <= @report_end_date
)

Query is running:   0%|          |

""


In [6]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Delivered_v4_Oct22_test` AS ( -- everyone who have received emails in the month
    SELECT DISTINCT adobe_tracking_id 
    FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
    WHERE event_name = 'Email Deliveries' 
    AND event_date BETWEEN @report_start_date and @report_end_date
    AND LOWER(campaign_name) NOT LIKE 'transactional%' -- Exclude transactional emails
)

Query is running:   0%|          |

""


In [7]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Holdout_v4_Oct22_test` AS ( 
    SELECT  DISTINCT holdout.adobe_tracking_id
    FROM
    (
        SELECT  DISTINCT TrackingId AS adobe_tracking_id
        FROM `nbcu-ds-prod-001.PeacockDataMartMarketingGold.HOLDOUT_GROUP`
        WHERE cohort = format_timestamp('%B%Y', DATETIME_TRUNC(@report_start_date, QUARTER)) -- get cohort name as month of quarter start + year
        AND Hold_Out_Type_Current = 'Owned Email Holdout'
        AND DATE(TIMESTAMP(RegistrationDate), 'America/New_York') <= @report_end_date
    ) holdout
    -- Exclude those who are assigned to Email Holdout but actually received emails in holdout period
    LEFT JOIN (
        SELECT DISTINCT adobe_tracking_id 
        FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
        WHERE event_name = 'Email Deliveries' 
        AND event_date BETWEEN DATETIME_TRUNC(@report_start_date, QUARTER) and @report_end_date
        AND LOWER(campaign_name) NOT LIKE 'transactional%' -- Exclude transactional emails
    ) received
    ON holdout.adobe_tracking_id = received.adobe_tracking_id
    WHERE received.adobe_tracking_id IS NULL
)

Query is running:   0%|          |

""


In [8]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Channel_Qualifier_v4_Oct22_test` AS 

-- Engagement: Deliveries 4 months before start of the holdout period, defined as start of quarter
SELECT DISTINCT adobe_tracking_id AS aid 
FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
WHERE event_name = 'Email Deliveries'
AND event_date BETWEEN DATE_SUB(DATETIME_TRUNC(@report_start_date, QUARTER), INTERVAL 4 MONTH) AND @report_end_date
AND lower(campaign_name) NOT LIKE 'transactional%' -- Exclude transactional emails

UNION ALL

-- New users joining after 4 months before start of the cohort period
SELECT DISTINCT adobe_tracking_id AS aid
FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
WHERE registration_date BETWEEN DATE_SUB(DATETIME_TRUNC(@report_start_date, QUARTER), INTERVAL 4 MONTH) AND @report_end_date

Query is running:   0%|          |

""


### Output Base Tables

In [9]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Measurement_Audience_v4_Oct22_test` AS

SELECT  distinct delivered_and_holdout.adobe_tracking_id AS aid
       ,cohort
       ,user.account_type
       ,user.account_tenure
FROM (
    SELECT *, 'Email_Targeted' as cohort from `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Delivered_v4_Oct22_test`
    UNION ALL
    SELECT *, 'Holdout' as cohort from `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Holdout_v4_Oct22_test`
) delivered_and_holdout

-- Include only those who received email in the current reporting period or are in holdout
INNER JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Channel_Qualifier_v4_Oct22_test` qualified
ON delivered_and_holdout.adobe_tracking_id = qualified.aid

-- for after 2021/july, email channel only, take out all abandon MAAs
INNER JOIN
    (
        SELECT  DISTINCT adobe_tracking_id
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES`
        WHERE report_date = @report_end_date
        AND date_of_last_view IS NOT NULL
    ) abandon_maa
ON delivered_and_holdout.adobe_tracking_id = abandon_maa.adobe_tracking_id

--add attribute: account_type at the end of the reporting period
INNER JOIN
    (
        SELECT  DISTINCT adobe_tracking_id
            ,account_type
            ,account_tenure
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
        WHERE report_date = @report_end_date 
    ) user
ON delivered_and_holdout.adobe_tracking_id = user.adobe_tracking_id

-- exclude unsubscribed
LEFT JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Unsubs_v4_Oct22_test` email_unsubs
ON delivered_and_holdout.adobe_tracking_id = email_unsubs.adobe_tracking_id
WHERE email_unsubs.adobe_tracking_id IS NULL

Query is running:   0%|          |

""


### Audience total size

In [19]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
SELECT cohort, count(distinct aid) from `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Measurement_Audience_v4_Oct22_test` group by 1

Query is running:   0%|          |

Downloading:   0%|          |

,cohort,f0_
0,Email_Targeted,40298993
1,Holdout,2018322


In [40]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
WITH 
base as (
    SELECT aid, cohort, paying_account_flag, billing_cycle
    from `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Measurement_Audience_v4_Oct22_test` a
    --add attribute: account_type at the end of the reporting period
    INNER JOIN
        (
            SELECT  DISTINCT adobe_tracking_id
                ,paying_account_flag
                ,billing_cycle
            FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
            WHERE report_date = @report_end_date 
        ) user
    ON a.aid = user.adobe_tracking_id
)
, EOM_Paid_Churn_Denom AS (
    SELECT  distinct adobe_tracking_id
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_CHURN`
    WHERE base_date = @report_end_date -- churn date would be on 11-30
    AND entitlement = 'Paid'
)
, EOM_Paid_Churn_Num AS (
    SELECT  distinct adobe_tracking_id
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_CHURN`
    WHERE base_date = @report_end_date
    AND entitlement = 'Paid'
    AND Churn_flag = 'Churn'
)
, cte1 as (
    SELECT cohort
        ,aid
        ,EOM_Paid_Churn_Denom.adobe_tracking_id  AS Churn_Denom
        ,EOM_Paid_Churn_Num.adobe_tracking_id    AS Churn_Num
    FROM base a
    LEFT JOIN EOM_Paid_Churn_Denom
        on a.aid = EOM_Paid_Churn_Denom.adobe_tracking_id
    LEFT JOIN EOM_Paid_Churn_Num
        on EOM_Paid_Churn_Denom.adobe_tracking_id = EOM_Paid_Churn_Num.adobe_tracking_id
    WHERE paying_account_flag = 'Paying'
    --and billing_cycle = 'MONTHLY'
)
, cte2 as (
    SELECT cohort
        ,COUNT(distinct Churn_Denom )   AS EOM_Paid_Churn_Denom_Targeted
        ,COUNT(distinct Churn_Num )     AS EOM_Paid_Churn_Num_Targeted
    FROM cte1
    group by 1
)
SELECT  cohort
       ,EOM_Paid_Churn_Denom_Targeted
       ,EOM_Paid_Churn_Num_Targeted
       ,safe_divide(EOM_Paid_Churn_Num_Targeted,EOM_Paid_Churn_Denom_Targeted) AS churn_rate
FROM cte2

Query is running:   0%|          |

Downloading:   0%|          |

,cohort,EOM_Paid_Churn_Denom_Targeted,EOM_Paid_Churn_Num_Targeted,churn_rate
0,Email_Targeted,10233978,595538,0.058192
1,Holdout,435022,26846,0.061712


Churn metrics channel version results

# Wes version

In [13]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params

CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.churn_oct22_wes` AS

WITH EmailDeliveryStatus AS (
    SELECT  distinct @report_start_date Month_Year
        ,adobe_tracking_id aid
        ,event_name
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
    WHERE date(eventTimestamp) BETWEEN @report_start_date AND @report_end_date
    AND event_name in('Email Deliveries')
    AND lower(campaign_name) not like '%transactional%' -- remove transactional emails 
    UNION ALL
    SELECT  distinct @report_start_date Month_Year
        ,adobe_tracking_id aid
        ,event_name -- identify if the last action a sub took was to unsubscribe
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
    WHERE date(eventTimestamp) < @report_end_date
    AND event_name in('Email Unsubscribes') 
)
SELECT  @report_start_date Month_Year
       ,l.aid
       ,l.billing_platform
       ,l.billing_cycle
       ,tenure_paid_lens
       ,CASE WHEN abandoned_ids is not null THEN "x"  ELSE null END abandoned_flag
       ,marketing_status.category
       ,CASE WHEN l.billing_platform = 'NBCU' THEN 'Direct'  ELSE 'IAP' END grouped_billing_platform
       ,CASE WHEN l.paying_account_flag != r.paying_account_flag THEN 'Downgrade'  ELSE 'No Change' END change_flag
       ,CASE WHEN l.voucher_partner is null THEN "Not On Voucher"  ELSE "On Voucher" END voucher_flag
       ,CASE WHEN ho.aid is not null THEN "Holdout"
             WHEN ho1.hid is not null THEN "Exclude"
             WHEN l.aid is not null THEN 'Targetable' END Audience
FROM
(
	SELECT  adobe_tracking_id aid
	       ,household_id hid
	       ,billing_platform
	       ,paying_account_flag
	       ,billing_cycle
	       ,tenure_paid_lens
	       ,voucher_partner
	FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
	WHERE report_date = @report_end_date
	AND paying_account_flag = 'Paying' 
) l

-- retrieve abandoned maas
LEFT JOIN
(
	SELECT  adobe_tracking_id abandoned_ids
	FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES`
	WHERE report_date = @report_end_date
	AND date_of_last_view is null 
) a
ON l.aid = a.abandoned_ids

-- retrieve thier marketing status: where they are unsubscribed or received an email in the last 4 months
LEFT JOIN
(
	SELECT  aid
	       ,event_name category -- identify if the last action a sub took was to unsubscribe
	FROM EmailDeliveryStatus
	WHERE Month_Year = @report_start_date 
) marketing_status
ON l.aid = marketing_status.aid

-- retrieve status 1 month later
LEFT JOIN
(
	SELECT  adobe_tracking_id
	       ,billing_platform
	       ,paying_account_flag
	FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
	WHERE report_date = '2022-11-30' 
) r
ON l.aid = r.adobe_tracking_id

-- retrieve members of holdout group
LEFT JOIN
(
	SELECT  aid
	FROM `nbcu-ds-sandbox-a-001.wes_crm_sandbox.Q42022_Holdout`
) ho
ON l.aid = ho.aid

--retrieve the rest of the global hold out that doesnt fit holdout definition. exclude these users from the analysis
LEFT JOIN
(
	SELECT  HouseholdId hid
	FROM `nbcu-ds-prod-001.PeacockDataMartMarketingGold.HOLDOUT_GROUP`
	WHERE cohort = 'October2022'
	AND Hold_Out_Type_Current = 'Owned Email Holdout' -- Exclude those who are assigned to Email Holdout but actually received emails
	AND TrackingId not in(
	SELECT  aid
	FROM `nbcu-ds-sandbox-a-001.wes_crm_sandbox.Q42022_Holdout` )
) ho1
ON l.hid = ho1.hid


Query is running:   0%|          |

""


In [39]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
SELECT  Month_Year
       ,audience
       ,COUNT(distinct aid) paid_maas
       ,COUNT(distinct CASE WHEN change_flag = 'Downgrade' THEN aid else null end) churns
       ,COUNT(distinct CASE WHEN change_flag = 'Downgrade' THEN aid else null end) / COUNT(distinct aid) as churn_rate
FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.churn_oct22_wes`
WHERE audience IN ('Targetable', 'Holdout')
GROUP BY  1,2

Query is running:   0%|          |

Downloading:   0%|          |

,Month_Year,audience,paid_maas,churns,churn_rate
0,2022-10-01,Targetable,16727799,1018284,0.060874
1,2022-10-01,Holdout,408121,23047,0.056471


## Difference Comparison

In [ ]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
select count(adobe_tracking_id) 
from `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Holdout_v4_Oct22_test` a
join `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Channel_Qualifier_v4_Oct22_test` b
on a.adobe_tracking_id = b.aid

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,2755643


In [ ]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
select count(aid) from `nbcu-ds-sandbox-a-001.wes_crm_sandbox.Q42022_Holdout`

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,1962559


## Edited version

In [69]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params

CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.churn_oct22_wes_edit` AS

WITH EmailDelivered AS (
    SELECT  distinct @report_start_date Month_Year
        ,adobe_tracking_id aid
    FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Delivered_v4_Oct22_test` a
	join `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Channel_Qualifier_v4_Oct22_test` b
	on a.adobe_tracking_id = b.aid
)
SELECT  @report_start_date Month_Year
       ,l.aid
       ,l.billing_platform
       ,l.billing_cycle
       ,tenure_paid_lens
	   ,CASE WHEN EmailDelivered.aid is not null then 'x' else null end as delivered_flag
       ,CASE WHEN abandoned_ids is not null THEN "x"  ELSE null END abandoned_flag
       ,case when unsub_status.aid is not null then 'x' else null end as unsub_flag
       ,CASE WHEN l.billing_platform = 'NBCU' THEN 'Direct'  ELSE 'IAP' END grouped_billing_platform
       ,CASE WHEN l.paying_account_flag != r.paying_account_flag THEN 'Downgrade'  ELSE 'No Change' END change_flag
       ,CASE WHEN l.voucher_partner is null THEN "Not On Voucher"  ELSE "On Voucher" END voucher_flag
       ,CASE WHEN ho.aid is not null THEN "Holdout"
             WHEN ho1.hid is not null THEN "Exclude"
             WHEN l.aid is not null THEN 'Targetable' END Audience
FROM
(
	SELECT  adobe_tracking_id aid
	       ,household_id hid
	       ,billing_platform
	       ,paying_account_flag
	       ,billing_cycle
	       ,tenure_paid_lens
	       ,voucher_partner
	FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
	WHERE report_date = @report_end_date
	AND paying_account_flag = 'Paying' 
) l

LEFT JOIN EmailDelivered on l.aid = EmailDelivered.aid

-- retrieve abandoned maas
LEFT JOIN
(
	SELECT  adobe_tracking_id abandoned_ids
	FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES`
	WHERE report_date = @report_end_date
	AND date_of_last_view is null 
) a
ON l.aid = a.abandoned_ids

-- retrieve thier marketing status: where they are unsubscribed or received an email in the last 4 months
LEFT JOIN
(
	SELECT  distinct @report_start_date Month_Year
        ,adobe_tracking_id aid
        ,event_name -- identify if the last action a sub took was to unsubscribe
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
    WHERE date(eventTimestamp) < @report_end_date
    AND event_name in('Email Unsubscribes') 
) unsub_status
ON l.aid = unsub_status.aid

-- retrieve status 1 month later
LEFT JOIN
(
	SELECT  adobe_tracking_id
	       ,billing_platform
	       ,paying_account_flag
	FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
	WHERE report_date = '2022-11-30' 
) r
ON l.aid = r.adobe_tracking_id

-- retrieve members of holdout group
LEFT JOIN
(
	select aid
	from `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Holdout_v4_Oct22_test` a
	join `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Channel_Qualifier_v4_Oct22_test` b
	on a.adobe_tracking_id = b.aid
) ho
ON l.aid = ho.aid

--retrieve the rest of the global hold out that doesnt fit holdout definition. exclude these users from the analysis
LEFT JOIN
(
	SELECT  HouseholdId hid
	FROM `nbcu-ds-prod-001.PeacockDataMartMarketingGold.HOLDOUT_GROUP`
	WHERE cohort = 'October2022'
	AND Hold_Out_Type_Current = 'Owned Email Holdout' -- Exclude those who are assigned to Email Holdout but actually received emails
	AND TrackingId not in(
	SELECT  aid
	FROM `nbcu-ds-sandbox-a-001.wes_crm_sandbox.Q42022_Holdout` )
) ho1
ON l.hid = ho1.hid


Query is running:   0%|          |

""


In [72]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params

SELECT
    Month_Year,
    audience,
    delivered_flag,
    count(distinct aid) as denom,
    count(distinct case when change_flag = 'Downgrade' then aid else null end) as churn,
    count(distinct case when change_flag = 'Downgrade' then aid else null end) / count(distinct aid) as churn_rate
FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.churn_oct22_wes_edit`
WHERE abandoned_flag is null
AND unsub_flag is null
--AND billing_cycle = 'MONTHLY'
AND audience in ('Targetable', 'Holdout')
group by 1,2,3

Query is running:   0%|          |

Downloading:   0%|          |

,Month_Year,audience,delivered_flag,denom,churn,churn_rate
0,2022-10-01,Targetable,x,10233689,595557,0.058196
1,2022-10-01,Targetable,None,5118643,319520,0.062423
2,2022-10-01,Holdout,None,435031,26848,0.061715


Results

## Edit version 2

In [84]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params

CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.churn_oct22_wes_edit_v2` AS

WITH EmailDeliveryStatus AS (
    SELECT  distinct @report_start_date Month_Year
        ,adobe_tracking_id aid
        ,event_name
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
    WHERE date(eventTimestamp) BETWEEN @report_start_date AND @report_end_date
    AND event_name in('Email Deliveries')
    AND lower(campaign_name) not like '%transactional%' -- remove transactional emails 
    UNION ALL
    SELECT  distinct @report_start_date Month_Year
        ,adobe_tracking_id aid
        ,event_name -- identify if the last action a sub took was to unsubscribe
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
    WHERE date(eventTimestamp) < @report_end_date
    AND event_name in('Email Unsubscribes') 
)
,Qualified as (
	-- Engagement: Deliveries 4 months before start of the holdout period, defined as start of quarter
	SELECT DISTINCT adobe_tracking_id AS aid 
	FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
	WHERE event_name = 'Email Deliveries'
	AND event_date BETWEEN DATE_SUB(DATETIME_TRUNC(@report_start_date, QUARTER), INTERVAL 4 MONTH) AND @report_end_date
	AND lower(campaign_name) NOT LIKE 'transactional%' -- Exclude transactional emails

	UNION ALL

	-- New users joining after 4 months before start of the cohort period
	SELECT DISTINCT adobe_tracking_id AS aid
	FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
	WHERE registration_date BETWEEN DATE_SUB(DATETIME_TRUNC(@report_start_date, QUARTER), INTERVAL 4 MONTH) AND @report_end_date
)
,Holdout AS (
	SELECT  DISTINCT holdout.adobe_tracking_id
    FROM
    (
        SELECT  DISTINCT TrackingId AS adobe_tracking_id
        FROM `nbcu-ds-prod-001.PeacockDataMartMarketingGold.HOLDOUT_GROUP`
        WHERE cohort = format_timestamp('%B%Y', DATETIME_TRUNC(@report_start_date, QUARTER)) -- get cohort name as month of quarter start + year
        AND Hold_Out_Type_Current = 'Owned Email Holdout'
        AND DATE(TIMESTAMP(RegistrationDate), 'America/New_York') <= @report_end_date
    ) holdout
    -- Exclude those who are assigned to Email Holdout but actually received emails in holdout period
    LEFT JOIN (
        SELECT DISTINCT adobe_tracking_id 
        FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
        WHERE event_name = 'Email Deliveries' 
        AND event_date BETWEEN DATETIME_TRUNC(@report_start_date, QUARTER) and @report_end_date
        AND LOWER(campaign_name) NOT LIKE 'transactional%' -- Exclude transactional emails
    ) received
    ON holdout.adobe_tracking_id = received.adobe_tracking_id
    WHERE received.adobe_tracking_id IS NULL
)
SELECT  @report_start_date Month_Year
       ,l.aid
       ,l.billing_platform
       ,l.billing_cycle
       ,tenure_paid_lens
	   ,CASE WHEN q.aid is not null then 'x' else null end as qualified_flag
       ,CASE WHEN abandoned_ids is not null THEN "x"  ELSE null END abandoned_flag
       ,marketing_status.category
       ,CASE WHEN l.billing_platform = 'NBCU' THEN 'Direct'  ELSE 'IAP' END grouped_billing_platform
       ,CASE WHEN l.paying_account_flag != r.paying_account_flag THEN 'Downgrade'  ELSE 'No Change' END change_flag
       ,CASE WHEN l.voucher_partner is null THEN "Not On Voucher"  ELSE "On Voucher" END voucher_flag
       ,CASE WHEN ho.aid is not null THEN "Holdout"
             WHEN ho1.hid is not null THEN "Exclude"
             WHEN l.aid is not null THEN 'Targetable' END Audience
FROM
(
	SELECT  adobe_tracking_id aid
	       ,household_id hid
	       ,billing_platform
	       ,paying_account_flag
	       ,billing_cycle
	       ,tenure_paid_lens
	       ,voucher_partner
	FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
	WHERE report_date = @report_end_date
	AND paying_account_flag = 'Paying' 
) l

LEFT JOIN Qualified q
ON l.aid = q.aid

-- retrieve abandoned maas
LEFT JOIN
(
	SELECT  adobe_tracking_id abandoned_ids
	FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES`
	WHERE report_date = @report_end_date
	AND date_of_last_view is null 
) a
ON l.aid = a.abandoned_ids

-- retrieve thier marketing status: where they are unsubscribed or received an email in the last 4 months
LEFT JOIN
(
	SELECT  aid
	       ,event_name category -- identify if the last action a sub took was to unsubscribe
	FROM EmailDeliveryStatus
	WHERE Month_Year = @report_start_date 
) marketing_status
ON l.aid = marketing_status.aid

-- retrieve status 1 month later
LEFT JOIN
(
	SELECT  adobe_tracking_id
	       ,billing_platform
	       ,paying_account_flag
	FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
	WHERE report_date = '2022-11-30' 
) r
ON l.aid = r.adobe_tracking_id

-- retrieve members of holdout group
LEFT JOIN (
	select aid 
	from Holdout
	join Qualified
	on Holdout.adobe_tracking_id = Qualified.aid
) ho
ON l.aid = ho.aid

--retrieve the rest of the global hold out that doesnt fit holdout definition. exclude these users from the analysis
LEFT JOIN
(
	SELECT  HouseholdId hid
	FROM `nbcu-ds-prod-001.PeacockDataMartMarketingGold.HOLDOUT_GROUP`
	WHERE cohort = 'October2022'
	AND Hold_Out_Type_Current = 'Owned Email Holdout' -- Exclude those who are assigned to Email Holdout but actually received emails
	AND TrackingId not in(
	SELECT  aid
	FROM `nbcu-ds-sandbox-a-001.wes_crm_sandbox.Q42022_Holdout` )
) ho1
ON l.hid = ho1.hid


Query is running:   0%|          |

""


In [85]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params

SELECT
    Month_Year,
    audience,
    count(distinct aid) as denom,
    count(distinct case when change_flag = 'Downgrade' then aid else null end) as churn,
    count(distinct case when change_flag = 'Downgrade' then aid else null end) / count(distinct aid) as churn_rate
FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.churn_oct22_wes_edit_v2`
WHERE abandoned_flag is null
AND category != 'Email Unsubscribes'
--AND billing_cycle = 'MONTHLY'
AND (audience = 'Targetable' and qualified_flag is not null) or (audience = 'Holdout')
group by 1,2

Query is running:   0%|          |

Downloading:   0%|          |

,Month_Year,audience,denom,churn,churn_rate
0,2022-10-01,Targetable,10284079,600910,0.058431
1,2022-10-01,Holdout,445370,27766,0.062344


results